In [1]:
import numpy as np, curvedsky as cs, plottools as pl, cmb as CMB, cosmology, healpy as hp, local
from matplotlib.pyplot import *
import warnings
warnings.filterwarnings("ignore")
ac2rad  = np.pi/10800.
deg2rad = np.pi/180.

In [2]:
def noise(lcl,sigp,theta,nu,name,dalpha=1.):
    lmax = len(lcl[0,:]) - 1
    L = np.linspace(0,lmax,lmax+1)
    nl = np.zeros((4,lmax+1))
    nl[0,:] = .5*(sigp*ac2rad/CMB.Tcmb)**2*np.exp(L*(L+1)*(theta*ac2rad)**2/8./np.log(2.))
    nl[1,:] = 2*nl[0,:]
    nl[2,:] = 2*nl[0,:]
    if nu!=0.:
        fEE, fBB = local.foreground(nu,L,name)
        nl[1,:] += fEE
        nl[2,:] += fBB
    Lcl = lcl.copy()
    Lcl[2,:] *= dalpha
    return Lcl + nl

In [3]:
def rec(Lmax,rlmin,rlmax,lcl,ocl):
    Ag = {}
    #Ag['TE'], __ = cs.norm_ilens.qte(Lmax,rlmin,rlmax,lcl[3,:],ocl[0,:],ocl[1,:])
    #Ag['EE'], __ = cs.norm_ilens.qee(Lmax,rlmin,rlmax,lcl[1,:],ocl[1,:])
    Ag['TB'] = cs.norm_imag.qtb('lens',Lmax,rlmin,rlmax,lcl[3,:],ocl[0,:],ocl[2,:])[0]
    Ag['EB'] = cs.norm_imag.qeb('lens',Lmax,rlmin,rlmax,lcl[1,:],ocl[1,:],ocl[2,:])[0]
    Ag['BB'] = cs.norm_imag.qbb('lens',Lmax,rlmin,rlmax,lcl[1,:],ocl[2,:])[0]
    return Ag

In [4]:
def snr_ilens(Ag,ucl,L,corr=1.):
    Nl = 1./(1./Ag['EB']+1./Ag['TB']+1./Ag['BB'])
    #Nl = 1./(1./Ag['EB']+1./Ag['TB']+1./Ag['BB']+1./Ag['EE']+1./Ag['TE'])
    SN = (2*L+1.)*ucl[3]/Nl * corr**2
    sn = deg2rad * np.sqrt( np.sum(SN[2:]) )
    return sn

In [5]:
def make_cov(EE,BB,EB,BE):
    return np.array( [ [ EE, EB ], [BE, BB ] ] )

In [6]:
def Fij_eb(lcl,ocl,lmin,lmax,nu,name,alpha=0.,beta=.35):
    pa = deg2rad * alpha
    pb = deg2rad * beta
    ln = lmax-lmin+1
    pn = 2
    L = np.linspace(lmin,lmax,ln)
    fEE, fBB = local.foreground(nu,L,name)
    lEE = lcl[1,lmin:lmax+1]
    lBB = lcl[2,lmin:lmax+1]
    oEE = ocl[1,lmin:lmax+1]
    oBB = ocl[2,lmin:lmax+1]
    oEB = 2*pa*(fEE-fBB) + 2*(pa+pb)*(lEE-lBB)
    # derivative
    dEBdp = np.zeros((pn,ln))
    dEBdp[0] = 2*(fEE-fBB) #+ 2*(lEE-lBB)
    dEBdp[1] = 2*(lEE-lBB)
    # covariance
    Cov = make_cov(oEE,oBB,oEB,oEB)
    iCov = np.array( [ np.linalg.inv(Cov[:,:,l]) for l in range(ln) ] )
    dlnCdp = np.zeros((2,2,ln,pn))
    for i in range(pn):
        dCov = make_cov(oEE*0.,oBB*0.,dEBdp[i],dEBdp[i])
        for l in range(ln):
            dlnCdp[:,:,l,i] = np.dot(iCov[l,:,:],dCov[:,:,l])
    Fl = cosmology.Fisher_Matrix(L,dlnCdp)
    F = np.sum( Fl, axis=2 ) 
    return F * deg2rad**2

In [7]:
def Fij_eb_old(lcl,ocl,lmin,lmax,nu,name,alpha=0.,beta=.35):
    pa = deg2rad * alpha
    pb = deg2rad * beta
    L = np.linspace(lmin,lmax,lmax-lmin+1)
    fEE, fBB = local.foreground(nu,L,name)
    lEE = lcl[1,lmin:lmax+1]
    lBB = lcl[2,lmin:lmax+1]
    oEE = ocl[1,lmin:lmax+1] 
    oBB = ocl[2,lmin:lmax+1] 
    oEB = 2*pa*(fEE-fBB) + 2*(pa+pb)*(lEE-lBB)
    rho2 = oEB**2/oEE*oBB
    fac = (2*L+1.)/(oEE*oBB) * (1+rho2)/(1-rho2)**2
    dEBda = 2*(fEE-fBB) + 2*(lEE-lBB)
    dEBdb = 2*(lEE-lBB)
    F = np.zeros((2,2))
    F[0,0] = np.sum( fac * dEBda * dEBda )
    F[0,1] = np.sum( fac * dEBdb * dEBda )
    F[1,1] = np.sum( fac * dEBdb * dEBdb )
    F[1,0] = F[0,1]*1.
    return F * deg2rad**2

In [8]:
def show_const(Fmatrix,text):
    sig  = np.sqrt(np.linalg.inv(Fmatrix).diagonal())
    usig = 1./np.sqrt(Fmatrix.diagonal())
    print(text+', const (arcmin):',sig*60.,usig*60.)
    return sig[0]*60, sig[1]*60, usig[0]*60

In [9]:
#Fold = Fij_eb_old(lcl,ocl,rlmin,Lmax,nu,alpha=0.,beta=.01)
#sig = np.sqrt(np.linalg.inv(Fold).diagonal())
#usig = 1./np.sqrt(Fold.diagonal())
#print(sig)
#print(usig)

In [10]:
Lmax  = 3000       # maximum multipole of output normalization
rlmin, rlmax = 50, Lmax  # CMB multipole range for reconstruction
L = np.linspace(0,Lmax,Lmax+1)
Lfac = L*(L+1)/2/np.pi
M = np.array([[1.,0],[1.,1.]])

In [12]:
ucl = CMB.read_camb_cls('../data/local/cosmo2017_10K_acc3_scalCls.dat',output='array')[:,:Lmax+1]
lcl = CMB.read_camb_cls('../data/local/cosmo2017_10K_acc3_lensedCls.dat',ftype='lens',output='array')[:,:Lmax+1]

In [13]:
name = 'Planck'
freqs, sigps, thetas = local.experiments(name)

In [14]:
sig = {}
for nu, sigp, theta in zip(freqs,sigps,thetas):
    print(nu,sigp,theta)
    sig[nu] = np.zeros(7)
    ocl = noise(lcl,sigp*2.,theta,nu,name)
    F = Fij_eb(lcl,ocl,rlmin,Lmax,nu,name,alpha=0.,beta=0.)
    __ = show_const( F, text=r'alpha,theta' )
    sig[0], sig[1], sig[2] = show_const( np.matmul(M.T,np.matmul(F,M)), text=r'alpha,beta' )
    ocl = noise(lcl,sigp,theta,nu,name)
    Ag  = rec(Lmax,rlmin,rlmax,lcl,ocl)
    snr = snr_ilens(Ag,ucl,L,corr=(0.4-0.9)/(2000.)*L+0.9)
    sig[3] = 60./snr
    print('sigma[deg]',sig[3]/60.)
    print('joint')
    Fpp = F + np.array([[0,0],[0,snr**2]])
    __ = show_const( Fpp, text=r'alpha,theta' )
    sig[4], sig[5], sig[6] = show_const( np.matmul(M.T,np.matmul(Fpp,M)), text=r'alpha,beta' )
    print('')
#np.save('sigma_'+name,sig)

30.0 424.26406871192853 33.2
alpha,theta, const (arcmin): [  68.85478814 2143.65939192] [  57.3595334  1785.77707984]
alpha,beta, const (arcmin): [  68.85478814 2182.50350235] [  56.33865071 1785.77707984]
sn 19.267103229984325
joint
alpha,theta, const (arcmin): [  60.141449   1017.50109711] [ 57.3595334  970.43535095]
alpha,beta, const (arcmin): [  60.141449   1037.16817974] [ 56.27186536 970.43535095]

44.0 424.26406871192853 28.0
alpha,theta, const (arcmin): [ 476.28709902 1407.77919561] [ 425.28100203 1257.01860951]
alpha,beta, const (arcmin): [ 476.28709902 1677.04093104] [ 356.99888766 1257.01860951]
sn 13.985778836119362
joint
alpha,theta, const (arcmin): [439.22606093 720.80647196] [425.28100203 697.92147127]
alpha,beta, const (arcmin): [439.22606093 933.14935993] [328.5061447  697.92147127]

70.0 271.5290039756343 13.0
alpha,theta, const (arcmin): [2426.41080579  197.29030285] [2365.59665818  192.34553358]
alpha,beta, const (arcmin): [2426.41080579 2477.78152616] [188.35772088